# Dynamic Topic Model

A dynamic topic model is 
we thought that this would be good to use because

In [1]:
import gensim
import pandas as pd
import numpy as np
from gensim.models import ldaseqmodel
from ast import literal_eval

In [2]:
df = pd.read_csv("../data/processed/formatted_df.csv").drop(columns = ['Unnamed: 0'])
df

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Name,Status,Description,References,Phase,Votes,Comments
0,CVE-1999-0001,Candidate,"['ipinputc', 'bsdderived', 'tcpip', 'implement...",BUGTRAQ:19981223 Re: CERT Advisory CA-98.13 - ...,Modified (20051217),"MODIFY(1) Frech | NOOP(2) Northcutt, W...",Christey> A Bugtraq posting indicates that the...
1,CVE-1999-0002,Entry,"['buffer', 'overflow', 'nfs', 'mountd', 'give'...",BID:121 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
2,CVE-1999-0003,Entry,"['execute', 'command', 'root', 'buffer', 'over...",BID:122 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
3,CVE-1999-0004,Candidate,"['mime', 'buffer', 'overflow', 'email', 'clien...",CERT:CA-98.10.mime_buffer_overflows | MS:M...,Modified (19990621),"ACCEPT(8) Baker, Cole, Collins, Dik, Landfi...","Frech> Extremely minor, but I believe e-mail i..."
4,CVE-1999-0005,Entry,"['arbitrary', 'command', 'execution', 'imap', ...",BID:130 | URL:http://www.securityfocus.com...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
166896,CVE-2021-46482,Candidate,"['jsish', 'v', 'discover', 'contain', 'heap', ...",MISC:https://github.com/pcmacdon/jsish/issues/66,Assigned (20220124),None (candidate not yet proposed),NaN
166897,CVE-2021-46483,Candidate,"['jsish', 'v', 'discover', 'contain', 'heap', ...",MISC:https://github.com/pcmacdon/jsish/issues/62,Assigned (20220124),None (candidate not yet proposed),NaN
166898,CVE-2021-46559,Candidate,"['firmware', 'moxa', 'tn', 'device', 'weak', '...",MISC:https://www.moxa.com/en/support/product-s...,Assigned (20220126),None (candidate not yet proposed),NaN
166899,CVE-2021-46560,Candidate,"['firmware', 'moxa', 'tn', 'device', 'allow', ...",MISC:https://www.moxa.com/en/support/product-s...,Assigned (20220126),None (candidate not yet proposed),NaN


Converts Description from string to list

In [3]:
df['Description'] = df['Description'].apply(literal_eval)

In [4]:
desc = df['Description']

In [5]:
vocab = gensim.corpora.Dictionary(desc)
doc_word_matrix = [vocab.doc2bow(doc) for doc in desc]

In [6]:
print(doc_word_matrix[3])

[(15, 1), (20, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)]


In [7]:
desc[19]

['arbitrary',
 'command',
 'execution',
 'buffer',
 'overflow',
 'countcgi',
 'wwwcount',
 'cgibin',
 'program']

To create an lda sequence model we need to count the number of documents in each year.  

We will use that the year of the vulnerability is the 4-8 characters of each of the CVE names. Therefore, we will extract these and count the instances for each one.

In [8]:
names = df['Name']
year = []
for instance in names:
    year.append(int(instance[4:8]))
year_count = []
for i in range(23):
    year_count.append(year.count(i+1999))
print(year_count)

[1541, 1237, 1535, 2350, 1498, 2633, 4586, 6858, 6340, 6971, 4887, 4992, 4587, 5401, 6120, 8279, 7916, 9200, 14319, 15481, 15269, 17784, 17117]


See how the chain variance affects things?

In [9]:
ldaseq = ldaseqmodel.LdaSeqModel(corpus = doc_word_matrix, id2word=vocab, time_slice=year_count)

/home/c/.local/lib/python3.8/site-packages/gensim/models/ldaseqmodel.py:297: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)


KeyboardInterrupt: 

In [ ]:
ldaseq.print_topics(time=0)

kf = KFold(n_splits=5, random_state=57, shuffle=True)
for split in kf.split(dftrain):
    print("round")
    #on the training data, do a 5-fold 
    train = dftrain.iloc[split[0]]
    test = dftrain.iloc[split[1]]
    xtrain = scale(train.iloc[:,:-2])
    xtest = scale(test.iloc[:,:-2])
    ytrain = train.iloc[:,-2]
    ytest = test.iloc[:,-2]
    #fit one of the k-folds 
    model_1 = svm.SVC(cache_size = 1000)
    instance = model_1.fit(xtrain,ytrain)
    temp_score_1 = instance.score(xtest,ytest)
    #if the new model works better than previous ones then save its parameters
    if temp_score_1 > score_1:
        score_1 = temp_score_1
        parameters_1 = instance
score = 0
best_params = []
#For each model test how it does in looking at new attacks
for j in range(4):
    print("j - ", j)
    instance = parameters[j]  
    scores = []
    for i in range(1,5):
        print(i)
        x = scale(df.loc[df['test'] == i].iloc[:,:-2])
        y = df.loc[df['test'] == i].iloc[:,-2]
        scores.append(instance.score(x,y))
        print(scores)
    if mean(scores) > score:
        score = mean(scores)
        best_params = parameters[j]
print(score)